In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import logging
from sklearn.preprocessing import LabelEncoder, StandardScaler

Task 2 - Prediction of store sales

2.1 Preprocessing

In [19]:
merged_data = pd.read_csv("./rossmann-store-sales/merged_data.csv", low_memory=False)

In [20]:
merged_data.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Day', 'WeekOfYear',
       'IsHoliday', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval',
       'CompetitionDistance_Binned', 'CityCenter'],
      dtype='object')

**Feature Engineering**

Date Based features 

In [21]:
merged_data.dtypes

Store                           int64
DayOfWeek                       int64
Date                           object
Sales                           int64
Customers                       int64
Open                            int64
Promo                           int64
StateHoliday                   object
SchoolHoliday                   int64
Year                            int64
Month                           int64
Day                             int64
WeekOfYear                      int64
IsHoliday                       int64
StoreType                      object
Assortment                     object
CompetitionDistance           float64
CompetitionOpenSinceMonth     float64
CompetitionOpenSinceYear      float64
Promo2                          int64
Promo2SinceWeek               float64
Promo2SinceYear               float64
PromoInterval                  object
CompetitionDistance_Binned     object
CityCenter                     object
dtype: object

In [22]:
merged_data['Date'] = pd.to_datetime(merged_data['Date'])

In [23]:
# Extracting Weekday Name
merged_data["WeekdayName"] = merged_data["Date"].dt.day_name()

In [24]:
merged_data["WeekdayName"].head(5)

0    Friday
1    Friday
2    Friday
3    Friday
4    Friday
Name: WeekdayName, dtype: object

In [25]:
# Weekday (1 if Monday-Friday, 0 otherwise)
merged_data["IsWeekday"] = merged_data["DayOfWeek"].apply(lambda x: 1 if x < 6 else 0)

In [26]:
# Weekend (1 if Saturday-Sunday, 0 otherwise)
merged_data["IsWeekend"] = merged_data["DayOfWeek"].apply(lambda x: 1 if x >= 6 else 0)

In [27]:
# Days Until Weekend (assuming weekend is Saturday and Sunday)
merged_data["DaysUntilWeekend"] = merged_data["DayOfWeek"].apply(lambda x: 6 - x if x < 6 else 0)

In [28]:
# Number of Days to the Next Holiday & Number of Days After Last Holiday
# Assuming holidays are marked in the 'IsHoliday' column
if "IsHoliday" in merged_data.columns:
    holiday_dates = merged_data.loc[merged_data["IsHoliday"] == 1, "Date"].unique()
    merged_data["DaysToNextHoliday"] = merged_data["Date"].apply(lambda x: min((holiday_dates - x).days[holiday_dates - x > np.timedelta64(0, 'D')], default=-1))
    merged_data["DaysSinceLastHoliday"] = merged_data["Date"].apply(lambda x: min((x - holiday_dates).days[x - holiday_dates > np.timedelta64(0, 'D')], default=-1))
else:
    merged_data["DaysToNextHoliday"] = -1
    merged_data["DaysSinceLastHoliday"] = -1

In [ ]:
# Beginning, Mid, and End of the Month Indicators
merged_data["MonthStart"] = (merged_data["Day"] <= 10).astype(int)
merged_data["MonthMid"] = ((merged_data["Day"] > 10) & (merged_data["Day"] <= 20)).astype(int)
merged_data["MonthEnd"] = (merged_data["Day"] > 20).astype(int)

In [30]:
merged_data.head(5)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,CityCenter,WeekdayName,IsWeekday,IsWeekend,DaysUntilWeekend,DaysToNextHoliday,DaysSinceLastHoliday,MonthStart,MonthMid,MonthEnd
0,1,5,2015-07-31,5263,555,1,1,0,1,2015,...,City Center,Friday,1,0,1,-1,5,0,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1,2015,...,City Center,Friday,1,0,1,-1,5,0,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1,2015,...,Non-City,Friday,1,0,1,-1,5,0,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,2015,...,City Center,Friday,1,0,1,-1,5,0,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1,2015,...,Non-City,Friday,1,0,1,-1,5,0,0,1


In [31]:
merged_data['DaysToNextHoliday'].unique()

array([-1,  1,  2,  3,  4,  5])

In [32]:
merged_data.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Day', 'WeekOfYear',
       'IsHoliday', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval',
       'CompetitionDistance_Binned', 'CityCenter', 'WeekdayName', 'IsWeekday',
       'IsWeekend', 'DaysUntilWeekend', 'DaysToNextHoliday',
       'DaysSinceLastHoliday', 'MonthStart', 'MonthMid', 'MonthEnd'],
      dtype='object')

In [33]:
if "CompetitionOpenSinceYear" in merged_data.columns and "CompetitionOpenSinceMonth" in merged_data.columns:
    # Convert to integers
    merged_data["CompetitionOpenSinceYear"] = merged_data["CompetitionOpenSinceYear"].astype(int)
    merged_data["CompetitionOpenSinceMonth"] = merged_data["CompetitionOpenSinceMonth"].astype(int)
    
    # Create competition start date
    merged_data["CompetitionStartDate"] = pd.to_datetime(
        merged_data["CompetitionOpenSinceYear"].astype(str) + "-" +
        merged_data["CompetitionOpenSinceMonth"].astype(str) + "-01",
        errors="coerce"
    )

In [34]:
# Identify if competition was active at the time of the sale
merged_data["IsCompetitionOpen"] = merged_data.apply(
        lambda row: 1 if pd.notna(row["CompetitionStartDate"]) and row["Date"] >= row["CompetitionStartDate"] else 0, axis=1)

In [35]:
# Calculate number of months since competition started
merged_data["CompetitionActiveMonths"] = merged_data.apply(
        lambda row: (row["Date"].year - row["CompetitionStartDate"].year) * 12 +
                    (row["Date"].month - row["CompetitionStartDate"].month) if row["IsCompetitionOpen"] == 1 else 0, axis=1
    )

In [36]:
# Calculate number of years since competition started
merged_data["CompetitionActiveYears"] = (merged_data["CompetitionActiveMonths"] / 12).astype(int)


In [37]:
# Drop the temporary column
merged_data.drop(columns=["CompetitionStartDate"], inplace=True)

Promo-Based Features

    Promo2Active (IsPromo2Active) → Binary (1 if Promo2 was active at that time, else 0).
    PromoRunningTime (PromoRunningWeeks) → Number of weeks since promo started.
    Seasonal Promo Indicator (IsSeasonalPromo) → Binary (1 if promo runs during seasonal months).

In [38]:
merged_data["Promo2SinceYear"] = merged_data["Promo2SinceYear"].astype(int)
merged_data["Promo2SinceWeek"] = merged_data["Promo2SinceWeek"].astype(int)

In [39]:
# Create Promo2 start date
merged_data["Promo2StartDate"] = pd.to_datetime(
    merged_data["Promo2SinceYear"].astype(str) + "-" +
    merged_data["Promo2SinceWeek"].astype(str) + "-1",  # Assuming week starts on Monday
    errors="coerce",
    format="%Y-%U-%w"
)

In [40]:
# Identify if Promo2 was active at the time of the sale
merged_data["IsPromo2Active"] = merged_data.apply(
    lambda row: 1 if pd.notna(row["Promo2StartDate"]) and row["Date"] >= row["Promo2StartDate"] else 0, axis=1)

In [41]:
# Calculate number of weeks since promo started
merged_data["PromoRunningWeeks"] = merged_data.apply(
    lambda row: (row["Date"] - row["Promo2StartDate"]).days // 7 if row["IsPromo2Active"] == 1 else 0, axis=1
)

In [42]:
# Drop the temporary column
merged_data.drop(columns=["Promo2StartDate"], inplace=True)

In [43]:
merged_data.head(5)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,DaysToNextHoliday,DaysSinceLastHoliday,MonthStart,MonthMid,MonthEnd,IsCompetitionOpen,CompetitionActiveMonths,CompetitionActiveYears,IsPromo2Active,PromoRunningWeeks
0,1,5,2015-07-31,5263,555,1,1,0,1,2015,...,-1,5,0,0,1,1,82,6,0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,2015,...,-1,5,0,0,1,1,92,7,1,278
2,3,5,2015-07-31,8314,821,1,1,0,1,2015,...,-1,5,0,0,1,1,103,8,1,225
3,4,5,2015-07-31,13995,1498,1,1,0,1,2015,...,-1,5,0,0,1,1,70,5,0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,2015,...,-1,5,0,0,1,1,3,0,0,0


Aggregated Sales & Customer Features

    AverageSalesPerCustomer (AvgSalesPerCustomer) → Sales per customer ratio.
    SalesPerPromoDay (SalesPerPromoDay) → Sales per promotional day.
    SalesLag1 (SalesLag1) → Sales of the previous day.
    SalesLag7 (SalesLag7) → Sales of the same weekday last week.
    SalesLag30 (SalesLag30) → Sales of the same day last month.
    RollingSalesMean7 (RollingSalesMean7) → Moving average sales for the last 7 days.
    RollingSalesMean30 (RollingSalesMean30) → Moving average sales for the last 30 days.
    SalesTrend (SalesTrend) → Difference between current sales and moving average.

In [44]:
# Sales per customer ratio
merged_data["AvgSalesPerCustomer"] = merged_data["Sales"] / merged_data["Customers"]
merged_data["AvgSalesPerCustomer"] = merged_data["AvgSalesPerCustomer"].fillna(0)  # Handle division by zero cases

In [45]:
# Sales per promotional day (only when Promo is active)
merged_data["SalesPerPromoDay"] = merged_data.apply(lambda row: row["Sales"] if row["Promo"] == 1 else 0, axis=1)

In [46]:
# Sales Lag Features
merged_data.sort_values(["Store", "Date"], inplace=True)
merged_data["SalesLag1"] = merged_data.groupby("Store")["Sales"].shift(1)
merged_data["SalesLag7"] = merged_data.groupby("Store")["Sales"].shift(7)
merged_data["SalesLag30"] = merged_data.groupby("Store")["Sales"].shift(30)

In [47]:
# Rolling Sales Mean Features
merged_data["RollingSalesMean7"] = merged_data.groupby("Store")["Sales"].rolling(7).mean().reset_index(level=0, drop=True)
merged_data["RollingSalesMean30"] = merged_data.groupby("Store")["Sales"].rolling(30).mean().reset_index(level=0, drop=True)

In [48]:
# Sales Trend (Difference between current sales and moving average)
merged_data["SalesTrend"] = merged_data["Sales"] - merged_data["RollingSalesMean7"]

In [49]:
merged_data.head(5)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,IsPromo2Active,PromoRunningWeeks,AvgSalesPerCustomer,SalesPerPromoDay,SalesLag1,SalesLag7,SalesLag30,RollingSalesMean7,RollingSalesMean30,SalesTrend
1016095,1,2,2013-01-01,0,0,0,0,a,1,2013,...,0,0,0.000000,0,NaN,NaN,NaN,NaN,NaN,NaN
1014980,1,3,2013-01-02,5530,668,1,0,0,1,2013,...,0,0,8.278443,0,0.0,NaN,NaN,NaN,NaN,NaN
1013865,1,4,2013-01-03,4327,578,1,0,0,1,2013,...,0,0,7.486159,0,5530.0,NaN,NaN,NaN,NaN,NaN
1012750,1,5,2013-01-04,4486,619,1,0,0,1,2013,...,0,0,7.247173,0,4327.0,NaN,NaN,NaN,NaN,NaN
1011635,1,6,2013-01-05,4997,635,1,0,0,1,2013,...,0,0,7.869291,0,4486.0,NaN,NaN,NaN,NaN,NaN


In [50]:
merged_data['RollingSalesMean7'].unique()

array([           nan,  3788.        ,  4585.14285714, ...,
       19940.14285714, 20487.42857143, 21613.28571429], shape=(83557,))

In [51]:
merged_data.dtypes

Store                                  int64
DayOfWeek                              int64
Date                          datetime64[ns]
Sales                                  int64
Customers                              int64
Open                                   int64
Promo                                  int64
StateHoliday                          object
SchoolHoliday                          int64
Year                                   int64
Month                                  int64
Day                                    int64
WeekOfYear                             int64
IsHoliday                              int64
StoreType                             object
Assortment                            object
CompetitionDistance                  float64
CompetitionOpenSinceMonth              int64
CompetitionOpenSinceYear               int64
Promo2                                 int64
Promo2SinceWeek                        int64
Promo2SinceYear                        int64
PromoInter

In [52]:
print(merged_data.isnull().sum())

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
Year                              0
Month                             0
Day                               0
WeekOfYear                        0
IsHoliday                         0
StoreType                         0
Assortment                        0
CompetitionDistance               0
CompetitionOpenSinceMonth         0
CompetitionOpenSinceYear          0
Promo2                            0
Promo2SinceWeek                   0
Promo2SinceYear                   0
PromoInterval                     0
CompetitionDistance_Binned        0
CityCenter                        0
WeekdayName                       0
IsWeekday                         0
IsWeekend                   

In [53]:
missing_cols = merged_data.columns[merged_data.isnull().sum() > 0]

In [54]:
missing_cols

Index(['SalesLag1', 'SalesLag7', 'SalesLag30', 'RollingSalesMean7',
       'RollingSalesMean30', 'SalesTrend'],
      dtype='object')

In [55]:
merged_data[missing_cols] = merged_data[missing_cols].fillna(0)

In [56]:
merged_data.isna().sum()

Store                         0
DayOfWeek                     0
Date                          0
Sales                         0
Customers                     0
Open                          0
Promo                         0
StateHoliday                  0
SchoolHoliday                 0
Year                          0
Month                         0
Day                           0
WeekOfYear                    0
IsHoliday                     0
StoreType                     0
Assortment                    0
CompetitionDistance           0
CompetitionOpenSinceMonth     0
CompetitionOpenSinceYear      0
Promo2                        0
Promo2SinceWeek               0
Promo2SinceYear               0
PromoInterval                 0
CompetitionDistance_Binned    0
CityCenter                    0
WeekdayName                   0
IsWeekday                     0
IsWeekend                     0
DaysUntilWeekend              0
DaysToNextHoliday             0
DaysSinceLastHoliday          0
MonthSta

Store Location Features

In [57]:
# Store Density: Count of stores in the same CityCenter region
merged_data["StoreDensity"] = merged_data.groupby("CityCenter")["Store"].transform("count")

In [58]:
merged_data["StoreDensity"]

1016095    459679
1014980    459679
1013865    459679
1012750    459679
1011635    459679
            ...  
5574       557530
4459       557530
3344       557530
2229       557530
1114       557530
Name: StoreDensity, Length: 1017209, dtype: int64

In [59]:
# Nearest Competitor Distance: Minimum CompetitionDistance in the same CityCenter
merged_data["NearestCompetitorDistance"] = merged_data.groupby("CityCenter")["CompetitionDistance"].transform("min")

In [60]:
# Store Foot Traffic: Estimated by multiplying Customers by Store Density (a proxy metric)
if "Customers" in merged_data.columns:
    merged_data["StoreFootTraffic"] = merged_data["Customers"] * merged_data["StoreDensity"]
else:
    merged_data["StoreFootTraffic"] = 0  # Default value if Customers column does not exist

In [61]:
merged_data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,SalesPerPromoDay,SalesLag1,SalesLag7,SalesLag30,RollingSalesMean7,RollingSalesMean30,SalesTrend,StoreDensity,NearestCompetitorDistance,StoreFootTraffic
1016095,1,2,2013-01-01,0,0,0,0,a,1,2013,...,0,0.0,0.0,0.0,0.0,0.0,0.0,459679,0.0,0
1014980,1,3,2013-01-02,5530,668,1,0,0,1,2013,...,0,0.0,0.0,0.0,0.0,0.0,0.0,459679,0.0,307065572
1013865,1,4,2013-01-03,4327,578,1,0,0,1,2013,...,0,5530.0,0.0,0.0,0.0,0.0,0.0,459679,0.0,265694462
1012750,1,5,2013-01-04,4486,619,1,0,0,1,2013,...,0,4327.0,0.0,0.0,0.0,0.0,0.0,459679,0.0,284541301
1011635,1,6,2013-01-05,4997,635,1,0,0,1,2013,...,0,4486.0,0.0,0.0,0.0,0.0,0.0,459679,0.0,291896165


In [62]:
merged_data.to_csv("./rossmann-store-sales/merged_data_final.csv", index=False)

In [63]:
merged_data.dtypes

Store                                  int64
DayOfWeek                              int64
Date                          datetime64[ns]
Sales                                  int64
Customers                              int64
Open                                   int64
Promo                                  int64
StateHoliday                          object
SchoolHoliday                          int64
Year                                   int64
Month                                  int64
Day                                    int64
WeekOfYear                             int64
IsHoliday                              int64
StoreType                             object
Assortment                            object
CompetitionDistance                  float64
CompetitionOpenSinceMonth              int64
CompetitionOpenSinceYear               int64
Promo2                                 int64
Promo2SinceWeek                        int64
Promo2SinceYear                        int64
PromoInter

In [64]:
merged_data.head(5)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,SalesPerPromoDay,SalesLag1,SalesLag7,SalesLag30,RollingSalesMean7,RollingSalesMean30,SalesTrend,StoreDensity,NearestCompetitorDistance,StoreFootTraffic
1016095,1,2,2013-01-01,0,0,0,0,a,1,2013,...,0,0.0,0.0,0.0,0.0,0.0,0.0,459679,0.0,0
1014980,1,3,2013-01-02,5530,668,1,0,0,1,2013,...,0,0.0,0.0,0.0,0.0,0.0,0.0,459679,0.0,307065572
1013865,1,4,2013-01-03,4327,578,1,0,0,1,2013,...,0,5530.0,0.0,0.0,0.0,0.0,0.0,459679,0.0,265694462
1012750,1,5,2013-01-04,4486,619,1,0,0,1,2013,...,0,4327.0,0.0,0.0,0.0,0.0,0.0,459679,0.0,284541301
1011635,1,6,2013-01-05,4997,635,1,0,0,1,2013,...,0,4486.0,0.0,0.0,0.0,0.0,0.0,459679,0.0,291896165
